# Transfer Learning

Image classification models have millions of parameters. Training them from scratch requires a lot of labeled training data and a lot of computing power. Transfer learning is a technique that shortcuts much of this by taking a piece of a model that has already been trained on a related task and reusing it in a new model.

**Transfer learning** is the process of applying existing machine learning models to scenarios for which they were not originally intended. This leveraging can save training time and extend the usefulness of existing machine learning models, models which may have had the available data and computation to have been trained for very long periods of time on very large datasets. If we train a model on a large set of data, we can then refine the result to be effective on our smaller amount of data. At least, that's the idea.

Documentation:

https://www.tensorflow.org/tutorials/images/transfer_learning

# Import **tensorflow** Library

Import the library and alias it:

In [ ]:
import tensorflow as tf

# GPU Hardware Accelerator

To vastly speed up processing, we can use the GPU available from the Google Colab cloud service. Colab provides a free Tesla K80 GPU of about 12 GB. It’s very easy to enable the GPU in a Colab notebook:

1.	click **Runtime** in the top left menu
2.	click **Change runtime** type from the drop-down menu
3.	choose **GPU** from the Hardware accelerator drop-down menu
4.	click **SAVE**

Verify that GPU is active:

In [ ]:
tf.__version__, tf.test.gpu_device_name()

# Pre-trained Models for Transfer Learning

If we don't have enough training data, it is often a good idea to reuse the lower layers of a pre-trained model. **Transfer learning** is the process of creating new AI models by fine-tuning previously trained neural networks. Instead of training a neural network from scratch, we can download a pretrained, open-source deep learning model and fine tune it for our own purpose.

To implement transfer learning, we reuse parts of a pre-trained model and change the final layer (or several layers) of the model. We then retrain those layers on our own dataset.

# Simple Transfer Learning with TensorFlow Hub

We model Flowers data by using pre-trained TF2 SavedModels from TensorFlow Hub for image feature extraction. The pre-trained models were trained on very large and general datasets.

We use two pre-trained TensorFlow Hub models to do transfer learning. **TensorFlow Hub** is a repository of trained machine learning models ready for fine-tuning and deployable anywhere. We begin with the MobileNet v2 pre-trained model. We then use the Inception v3 pre-trained model and compare results between the two.

Resources:

https://www.tensorflow.org/hub

https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub

# MobileNet v2 Example

Information about MobileNet and other pre-trained models is avaliable at the following URL:

https://tfhub.dev/s?module-type=image-feature-vector&q=tf2

## Load flowers as TFDS

Split 75% for train set, 15% for validation set, and 25% for test set:

In [ ]:
import tensorflow_datasets as tfds

(test, valid, train), info = tfds.load(
    'tf_flowers', as_supervised=True,
    split = ['train[:10%]', 'train[10%:25%]', 'train[25%:]'],
    with_info=True, try_gcs=True)

## Get Metadata

Display general information:

In [ ]:
info

Display number examples in data splits:

In [ ]:
num_train_img = info.splits['train[25%:]'].num_examples
num_valid_img = info.splits['train[10%:25%]'].num_examples
num_test_img = info.splits['train[:10%]'].num_examples
print ('train images:', num_train_img)
print ('valid images:', num_valid_img)
print ('test images:', num_test_img)

Calculate number of examples in data splits manually to verify:

In [ ]:
num_train_examples = 0
num_valid_examples = 0
num_test_examples = 0

for example in train:
  num_train_examples += 1

for example in valid:
  num_valid_examples += 1

for example in test:
  num_test_examples += 1

print('Total Number of Training Images: {}'\
      .format(num_train_examples))
print('Total Number of Validation Images: {}'\
      .format(num_valid_examples))
print('Total Number of Testing Images: {}'\
      .format(num_test_examples))

Get labels:

In [ ]:
class_labels = info.features['label'].names
class_labels

Get number of classes:

In [ ]:
num_classes = info.features['label'].num_classes
num_classes

## Display Examples

Display some examples with **show_examples**:

In [ ]:
fig = tfds.show_examples(train, info)

## Inspect Images

Display shapes to check images sizes:

In [ ]:
for i, example in enumerate(train.take(5)):
  print('Image {} shape: {} label: {}'\
        .format(i+1, example[0].shape,
                example[1]))

The images in the flowers dataset are not all the same size. So, we must resize images to a standard size to make them consumable by TensorFlow models.

## Build the Input Pipeline

Create a function to reformat all images to the resolution expected by MobileNet v2 (224, 224) and scale them. The function takes an 'image' and a 'label' as arguments and returns the new 'image' and corresponding 'label' in the desired form.

In [ ]:
def format_image(image, label):
  image = tf.image.resize(image, (224, 224)) /255.0
  return image, label

Map function to train, validation, and test sets. And, apply other transformations:

In [ ]:
BATCH_SIZE = 367

train_batches = train.shuffle(num_train_img//4).\
  map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_batches = valid.map(format_image).\
  batch(BATCH_SIZE).prefetch(1)

test_batches = test.map(format_image).\
  batch(BATCH_SIZE).prefetch(1)  

## Simple Transfer Learning with MobileNet-v2

We begin the process by creating  a feature_extractor. The partial model from TensorFlow Hub (without the final classification layer) is called a feature vector. Go to the TensorFlow Hub documentation (https://tfhub.dev/s?module-type=image-feature-vector) to see a list of available feature vectors.

To get information about MobileNet-v2:

https://www.tensorflow.org/api_docs/python/tf/keras/applications/MobileNetV2

Resource:

https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l06c03_exercise_flowers_with_transfer_learning_solution.ipynb

### Create a Feature Extractor

Create a feature_extractor using the MobileNet-v2 feature vector. A **feature extractor** is the partial model from TensorFlow Hub (without the final classification layer).

To see a list of available feature vectors, visit:

https://tfhub.dev/s?module-type=image-feature-vector&q=tf2

Click on one of them, read the documentation, and get the corresponding URL to get the feature vector.

Create the feature extractor:

In [ ]:
import tensorflow_hub as hub

piece1 = 'https://tfhub.dev/google/tf2-preview/'
piece2 = 'mobilenet_v2/feature_vector/4'
URL = piece1 + piece2
feature_extractor_mn = hub.KerasLayer(
    URL, input_shape=(224, 224, 3))

The feature extractor is now a partial MobileNet-v2 model.

## Freeze the Pretrained Model

Freeze the variables in the feature extractor layer, so that the training only modifies the final classifier layer:

In [ ]:
feature_extractor_mn.trainable = False

## Attach a Classification Head

Create a classification head to leverage the pre-trained model for the dataset, which consists of a simple sequential model that includes the pre-trained model and the new classification layer.

Import libraries:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

Clear previous models and generate seed:

In [ ]:
import numpy as np

tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

Build model:

In [ ]:
mobile_model = tf.keras.Sequential([
  feature_extractor_mn,
  Dropout(0.5),
  Dense(num_classes)])

Inspect model:

In [ ]:
mobile_model.summary()

## Compile

Compile **SparseCategoricalCrossentropy**:

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

mobile_model.compile(
  optimizer='adam',
  loss=SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

## Train

Train model on train and validation sets for six epochs:

In [ ]:
EPOCHS = 6

history = mobile_model.fit(
    train_batches, epochs=EPOCHS,
    validation_data=validation_batches)

We get good accuracy with just 6 epochs because MobileNet-v2 was carefully designed over a long time by experts and then trained on a massive dataset (ImageNet).

## Visualize Performance

Plot model performance:

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Make Predictions from Test Data

Predict on **test_batches**:

In [ ]:
predictions = mobile_model.predict(test_batches)

Test data is pure because we haven't seen it yet!

Display class labels:

In [ ]:
class_labels

### Inspect the First Prediction

Get the first prediction array:

In [ ]:
predictions[0]

The returned array is the raw prediction.

Use the np.argmax() function to get the prediction for the first image:

In [ ]:
predicted_id = np.argmax(predictions[0])
predicted_id

Convert the label to its class name:

In [ ]:
class_labels[predicted_id]

Get the actual labels from the first batch:

In [ ]:
for img, lbl in test_batches.take(1):
  print (lbl)

Get the first label:

In [ ]:
class_labels[lbl[0].numpy()]

The prediction is correct if the actual label matches the prediction.

### Inspect the First Batch of Predictions

Alternatively, we can convert *test_batches* to an iterator:

In [ ]:
image_batch, label_batch = next(iter(test_batches))

images = image_batch.numpy()
labels = label_batch.numpy()

class_labels[labels[0]]

Get the first batch from test_batches iterator with **next**, convert images and labels to NumPy, and display the first label.

Display labels from the first batch:

In [ ]:
labels

Convert the batch of labels to named labels:

In [ ]:
named_labels = [class_labels[labels[i]]
                for i, lbl in enumerate(range(BATCH_SIZE))]
named_labels

Get predictions from the first batch:

In [ ]:
predicted_batch = [np.argmax(predictions[i])
                   for i, _ in enumerate(range(BATCH_SIZE))]
predicted_batch

Convert predictions to named predictions:

In [ ]:
named_pred = [class_labels[predicted_batch[i]]
              for i, lbl in enumerate(range(BATCH_SIZE))]
named_pred

## Plot Model Predictions

The visualization shows actual images from the first test batch. If the prediction is correct, the title is blue. If not, the title is red. If the prediction is incorrect, the prediction is displayed along with the actual label in parentheses.

In [ ]:
plt.figure(figsize=(20,20))
for n in range(30):
  plt.subplot(6,5,n+1)
  plt.subplots_adjust(hspace = 0.3)
  plt.imshow(images[n])
  color = 'blue' if labels[n] == predicted_batch[n] else 'red'
  if labels[n] != predicted_batch[n]:
    t = named_pred[n].title() +\
        ' (' +named_labels[n].title() + ')'
  else:
    t = named_pred[n].title()
  plt.title(t, color=color)
  plt.axis('off')
  st = 'Model predictions (blue: correct, red: incorrect)'
_ = plt.suptitle(st)

# Perform Transfer Learning with the Inception Model

Use the Inception model to compare against MobileNet. To get the model, peruse https://tfhub.dev/s?module-type=image-feature-vector&q=tf2 and click on 'tf2-preview/inception_v3/feature_vector'. This feature vector corresponds to the Inception v3 model. Use transfer learning to create a CNN that uses Inception v3 as the pretrained model to classify the images from the Flowers dataset. Note that Inception takes as input images that are 299 x 299 pixels.

## Reformat Images and Create Batches

Recreate the **format_image** function to reformat images to the resolution expected by Inception v3 (299, 299), and scale them:

In [ ]:
def format_image(image, label):
  image = tf.image.resize(image, (299, 299)) / 255.0
  return image, label

## Build an Input Pipeline for Inception

Shuffle train data, reformat, batch, and prefetch train, validation, and test data:

In [ ]:
BATCH_SIZE = 367

train_im = train.shuffle(num_train_img//4).\
  map(format_image).batch(BATCH_SIZE).prefetch(1)

validation_im = valid.map(format_image).\
  batch(BATCH_SIZE).prefetch(1)

test_im = test.map(format_image).\
  batch(BATCH_SIZE).prefetch(1)

## Create a Feature Extractor

Create a feature_extractor using the Inception v3 feature vector. A **feature extractor** is the partial model from TensorFlow Hub (without the final classification layer).

To see a list of available feature vectors, visit:

https://tfhub.dev/s?module-type=image-feature-vector&q=tf2

Click on one of them, read the documentation, and get the corresponding URL to get the feature vector.

Create the feature extractor:

In [ ]:
piece1 = 'https://tfhub.dev/google/tf2-preview/'
piece2 = 'inception_v3/feature_vector/4'
URL = piece1 + piece2
feature_extractor_im = hub.KerasLayer(URL,
  input_shape=(299, 299, 3),
  trainable=False)

Freeze the pre-trained layers:

In [ ]:
feature_extractor_im.trainable = False

Clear and seed:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

## Create the Inception Model



We've already set the stage with MobileNet. So we just need to subtitute the Inception feature extractor:

In [ ]:
inception_model = tf.keras.Sequential([
  feature_extractor_im,
  Dropout(0.5),
  Dense(num_classes)])

## Compile

Compile with **SparseCategoricalCrossentropy(from_logits=True)**:

In [ ]:
inception_model.compile(
  optimizer='adam',
  loss=SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

## Train

Train model for six epochs:

In [ ]:
EPOCHS = 6

history = inception_model.fit(
    train_im, epochs=EPOCHS,
    validation_data=validation_im)

## Visualize

Visualize model performance:

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Predictions

Make predictions:

In [ ]:
im_predictions = inception_model.predict(test_im)

Get a batch of predictions and convert them to named predictions:

In [ ]:
im_pred_batch = [np.argmax(im_predictions[i])
                 for i, _ in enumerate(range(BATCH_SIZE))]
im_named_pred = [class_labels[im_pred_batch[i]]
                 for i, lbl in enumerate(range(BATCH_SIZE))]

Grab the first batch of images and labels from the test set:

In [ ]:
im_image_batch, im_label_batch = next(iter(test_im))

im_images = im_image_batch.numpy()
im_labels = im_label_batch.numpy()

Convert the labels to named labels:

In [ ]:
im_named_labels = [class_labels[im_labels[i]]
                   for i, lbl in enumerate(range(BATCH_SIZE))]

## Plot Model Predictions

Create a function to display predictions:

In [ ]:
def plot_pred(images, labels, named_labels, named_pred):
  plt.figure(figsize=(20,20))
  for n in range(30):
    plt.subplot(6,5,n+1)
    plt.subplots_adjust(hspace = 0.3)
    plt.imshow(images[n])
    color = 'blue' if named_labels[n] == named_pred[n] else 'red'
    if named_labels[n] != named_pred[n]:
      t = named_pred[n].title() +\
      ' (' +named_labels[n].title() + ')'
    else:
      t = named_pred[n].title()
    plt.title(t, color=color)
    plt.axis('off')
    st = 'Model predictions (blue: correct, red: incorrect)'
    _ = plt.suptitle(st)

Invoke the function:

In [ ]:
plot_pred(im_images, im_labels, im_named_labels, im_named_pred)